In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.filters import sobel
from skimage.morphology import binary_closing, binary_opening
from skimage import color

## Armazena as contagens manuais feitas por Helena e Rafael em um Dataframe 

In [4]:
contagem_manual = [
    {
        'imagem': 'imagem_01',
        'helena': 5,
        'rafael': 5
    },
    {
        'imagem': 'imagem_02',
        'helena': 4,
        'rafael': 4
    },
    {
        'imagem': 'imagem_03',
        'helena': 6,
        'rafael': 5
    },
    {
        'imagem': 'imagem_04',
        'helena': 6,
        'rafael': 5
    },
    {
        'imagem': 'imagem_05',
        'helena': 10,
        'rafael': 9
    },
    {
        'imagem': 'imagem_06',
        'helena': 9,
        'rafael': 8
    },
    {
        'imagem': 'imagem_07',
        'helena': 6,
        'rafael': 6
    },
    {
        'imagem': 'imagem_08',
        'helena': 5,
        'rafael': 5
    },
    {
        'imagem': 'imagem_09',
        'helena': 8,
        'rafael': 8
    },
    {
        'imagem': 'imagem_10',
        'helena': 5,
        'rafael': 5
    }
]

contagem_manual_df = pd.DataFrame(contagem_manual)
print(contagem_manual_df)

      imagem  helena  rafael
0  imagem_01       5       5
1  imagem_02       4       4
2  imagem_03       6       5
3  imagem_04       6       5
4  imagem_05      10       9
5  imagem_06       9       8
6  imagem_07       6       6
7  imagem_08       5       5
8  imagem_09       8       8
9  imagem_10       5       5


## Carrega os arquivos (imagens) da pasta Data 

In [ ]:
imagem_01 = io.imread('Data/imagem-01.jpg')
imagem_02 = io.imread('Data/imagem-02.jpg')
imagem_03 = io.imread('Data/imagem-03.jpg')
imagem_04 = io.imread('Data/imagem-04.jpg')
imagem_05 = io.imread('Data/imagem-05.jpg')
imagem_06 = io.imread('Data/imagem-06.jpg')
imagem_07 = io.imread('Data/imagem-07.jpg')
imagem_08 = io.imread('Data/imagem-08.jpg')
imagem_09 = io.imread('Data/imagem-09.jpg')
imagem_10 = io.imread('Data/imagem-10.jpg')